#  Coursera Applied Data Science Capstone Project

### This notebook is to be used for the Coursera Capstone Project

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto
First I define the URL and read it into a dataframe

In [2]:
!pip install requests
!pip install beautifulsoup4
!pip install lxml

import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
page  = requests.get(url)
page

<Response [200]>

In [5]:
data=page.text
soup = BeautifulSoup(data, 'html.parser')
#print(soup.prettify())
df=pd.read_html(url)
df=df[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


I then remove the rows which contain a "Not assigned" value in the Brorough column

In [6]:
#Pre-processing
#Removing "Not Assigned" Boroughs from dataframe
df=df[~df['Borough'].isin(["Not assigned"])]
df

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


I group my dataframe by Postcode and joining the Neighborhoods (separated by a comma) 

In [7]:
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood']= df['Neighbourhood'].str.join(', ')
df

,Postcode,Borough,Neighbourhood
0,M1X,Scarborough,Upper Rouge
1,M3L,North York,Downsview West
2,M5S,Downtown Toronto,"Harbord, University of Toronto"
3,M5M,North York,"Bedford Park, Lawrence Manor East"
4,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"
...,...,...,...
98,M4T,Central Toronto,"Moore Park, Summerhill East"
99,M6G,Downtown Toronto,Christie
100,M5C,Downtown Toronto,St. James Town
101,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"


If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough

In [8]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head()

,Postcode,Borough,Neighbourhood
0,M1X,Scarborough,Upper Rouge
1,M3L,North York,Downsview West
2,M5S,Downtown Toronto,"Harbord, University of Toronto"
3,M5M,North York,"Bedford Park, Lawrence Manor East"
4,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel"


In [9]:
df.shape

(103, 3)

## Then I use the csv file provided (since the geocoder package is very unreliable) to create the dataframe that includes the latitude and the longitude

In [10]:
df_geo = pd.read_csv('http://cocl.us/Geospatial_data')
df_geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [11]:
df_geo.rename(columns={'Postal Code':'Postcode'},inplace=True)
results = pd.merge(df,df_geo[['Postcode','Latitude','Longitude']],on='Postcode')
results

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1X,Scarborough,Upper Rouge,43.836125,-79.205636
1,M3L,North York,Downsview West,43.739015,-79.506944
2,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
3,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750
4,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
...,...,...,...,...,...
98,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
99,M6G,Downtown Toronto,Christie,43.669542,-79.422564
100,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
101,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944


### Now I explore and cluster the neighborhoods in Toronto

First I import the required libraries to create a map

In [12]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
#import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


### Create a map of Toronto with neighborhoods superimposed on top.

I create my dataframe to only contain Bouroughs where the word "Toronto" feautures

In [13]:
toronto_data=results[results['Borough'].str.contains('Toronto',na = False)]
toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
4,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
10,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
11,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
12,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
17,M5N,Central Toronto,Roselawn,43.711695,-79.416936
25,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
26,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
28,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
29,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


I then create a map with my new "Toronto" dataframe 

In [14]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now I want to cluster the neighborhoods in Toronto through k-means clustering

I remove the Postcode, Borough and Neighbourhood column and assign clusters based on the remaining column (i.e. the latitude and longitude)

In [16]:
k=5
toronto_clustering = toronto_data.drop(['Postcode','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
toronto_data.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_data

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,4,M5S,Downtown Toronto,"Harbord, University of Toronto",43.662696,-79.400049
4,0,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817
10,4,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
11,0,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
12,1,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
17,3,M5N,Central Toronto,Roselawn,43.711695,-79.416936
25,4,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
26,1,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
28,3,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
29,0,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


Then I create a map with the clustered data

In [17]:

# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters